# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll then also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

You will be able to:
* Use ANOVA for testing multiple pairwise comparisons
* Understand and explain the methodology behind ANOVA tests

In [4]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Loading the Data

Start by loading in the data stored in the file **ToothGrowth.csv**.

In [3]:
# Your code here
df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


In [6]:
type(df.supp[0])

str

In [14]:
df['supp2'] = df.supp
for i in range(len(df.supp)):
    if df.supp.loc[i] == 'VC':
        df.supp2.loc[i] = 0
    else:
        df.supp2.loc[i] = 1
df.head()

,len,supp,dose,supp2
0,4.2,VC,0.5,0
1,11.5,VC,0.5,0
2,7.3,VC,0.5,0
3,5.8,VC,0.5,0
4,6.4,VC,0.5,0


## Generating the ANOVA Table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage 

In [17]:
#Your code here
formula = 'len ~ dose + C(supp2)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp2)   205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Reading the Table

Make a brief comment regarding the statistics regarding the effect of supplement and dosage on tooth length.

#Your comment here
dose strength is a more robust predictor of tooth growth. Supplementation, while significant, has a weaker effect. 

## Comparing to T-Tests

Now that you've gotten a brief chance to interact with ANOVA, its interesting to compare the results to those from the t-tests you were just working with. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterwards, you'll conduct a t-test to compare the tooth length of these two different samples.

In [27]:
#Your code here
vc = df[df.supp2==0]['len']
oj = df[df.supp2==1]['len']

In [28]:
import flatiron_stats as fs

fs.p_value_welch_ttest(vc, oj, two_sided=True)

0.06063450788093405

Now compare a t-test between these two groups and print the associated two-sided p-value.

## A 2-Category ANOVA F-Test is Equivalent to a 2-Tailed t-Test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-Test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [29]:
#Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
#Compare the p-value to that of the t-test above. 
#They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

formula = 'len ~ C(supp2)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df         F    PR(>F)
C(supp2)   205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Generating Multiple T-Tests

While the 2-category ANOVA test is identical to a 2-tailed t-Test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [30]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2 sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [31]:
#Your code here; reuse your $t$-test code above to calculate the p-value for a 2-sided $t$-test
#for all combinations of the supplement-dose groups listed above. 
#(Since there isn't a control group, compare each group to every other group.)

from itertools import combinations

groups = [group[0] for group in df.groupby(['supp', 'dose'])['len']]
combos = combinations(groups, 2)
for combo in combos:
    supp1 = combo[0][0]
    dose1 = combo[0][1]
    supp2 = combo[1][0]
    dose2 = combo[1][1]
    sample1 = df[(df.supp == supp1) & (df.dose == dose1)]
    sample2 = df[(df.supp == supp2) & (df.dose == dose2)]
    p = fs.p_value_welch_ttest(sample1, sample2, two_sided=True)
    print(combo, p[0])

(('OJ', 0.5), ('OJ', 1.0)) 1.1102230246251565e-15
(('OJ', 0.5), ('OJ', 2.0)) 0.0
(('OJ', 0.5), ('VC', 0.5)) 2.5210389109631137e-08
(('OJ', 0.5), ('VC', 1.0)) 4.817632791609405e-05
(('OJ', 0.5), ('VC', 2.0)) 0.0
(('OJ', 1.0), ('OJ', 2.0)) 2.7423676985716128e-05
(('OJ', 1.0), ('VC', 0.5)) 0.0
(('OJ', 1.0), ('VC', 1.0)) 1.91500149071544e-11
(('OJ', 1.0), ('VC', 2.0)) 0.0007500878903570207
(('OJ', 2.0), ('VC', 0.5)) 0.0
(('OJ', 2.0), ('VC', 1.0)) 0.0
(('OJ', 2.0), ('VC', 2.0)) 0.9267849358960447
(('VC', 0.5), ('VC', 1.0)) 0.0
(('VC', 0.5), ('VC', 2.0)) 0.0
(('VC', 1.0), ('VC', 2.0)) 8.881784197001252e-16


/anaconda3/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:5193: RuntimeWarning: invalid value encountered in greater
  return df > 0
/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


## Summary

In this lesson, you examined the ANOVA technique to generalize A/B testing methods to multiple groups and factors.